In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
# Read csv
df_data = pd.read_csv(r'C:\Users\LEAND\Coding\knime-workspace\DMC2022\Leander\csv\complete_dataset_labeled_wLastPurchaseDates.csv', sep='|')
#df_data.head()

In [3]:
#df_data.info()

In [4]:
# Convert categories to data type category
#df_data["categories"] = df_data["categories"].astype("category")

# Preprocessing

In [5]:
# Drop columns
df_data.drop('purchaseDates', axis=1, inplace=True)
df_data.drop('date', axis=1, inplace=True)
df_data.drop('order', axis=1, inplace=True)

df_data.head()

,userID,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories,week,lastPurchaseDate
0,38769,3477,186,6,0,196,0,45,"[74, 4109, 3867, 803, 4053]",1,2222-03-03
1,42535,30474,193,10,3,229,3,132,"[3459, 3738, 679, 1628, 4072]",1,2222-03-03
2,42535,15833,1318,4,1,455,0,108,"[2973, 2907, 2749, 3357]",1,2222-03-03
3,42535,20131,347,4,0,291,3,44,"[30, 1515, 1760, 2932, 1287, 2615, 3727, 2450,...",1,2222-03-03
4,42535,4325,539,6,0,303,0,45,"[3104, 1772, 2029, 1274, 3915, 888, 1118, 3882...",1,2222-03-03


### Split datetime into separate columns

In [6]:
# Convert 'lastPurchaseDate' to datetime
df_data['lastPurchaseDate'] = pd.to_datetime(df_data['lastPurchaseDate'], format="%Y-%m-%d")

# Split 'date' into seperate columns
df_data['day'] = df_data['lastPurchaseDate'].dt.day
df_data['month'] = df_data['lastPurchaseDate'].dt.month
df_data['year'] = df_data['lastPurchaseDate'].dt.year

df_data.drop('lastPurchaseDate', axis=1, inplace=True)

In [7]:
"""
# Move new columns to the front
col = df_data.pop("year")
df_data.insert(1, col.name, col)

col = df_data.pop("month")
df_data.insert(2, col.name, col)

col = df_data.pop("day")
df_data.insert(3, col.name, col)
"""

'\n# Move new columns to the front\ncol = df_data.pop("year")\ndf_data.insert(1, col.name, col)\n\ncol = df_data.pop("month")\ndf_data.insert(2, col.name, col)\n\ncol = df_data.pop("day")\ndf_data.insert(3, col.name, col)\n'

In [8]:
# Show table
#df_data.head()

### Multi-Hot-Encoding for categories

In [9]:
# If you run out of memory while encoding the whole dataframe at once you can 
# split the dataframe into chunks beforehand 

df_data_chunk1 = df_data.iloc[:150000,:]
#df_chunk2 = df.iloc[500000:,:]

# Replace symbols in 'categories'
df_data_chunk1['categories'] = df_data_chunk1['categories'].str.replace("[", "")
df_data_chunk1['categories'] = df_data_chunk1['categories'].str.replace("]", "")

C:\Users\LEAND\AppData\Local\Temp/ipykernel_24140/231179628.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_data_chunk1['categories'] = df_data_chunk1['categories'].str.replace("[", "")
C:\Users\LEAND\AppData\Local\Temp/ipykernel_24140/231179628.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_chunk1['categories'] = df_data_chunk1['categories'].str.replace("[", "")
C:\Users\LEAND\AppData\Local\Temp/ipykernel_24140/231179628.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions wi

In [10]:
df_data_chunk1.head()

,userID,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories,week,day,month,year
0,38769,3477,186,6,0,196,0,45,"74, 4109, 3867, 803, 4053",1,3,3,2222
1,42535,30474,193,10,3,229,3,132,"3459, 3738, 679, 1628, 4072",1,3,3,2222
2,42535,15833,1318,4,1,455,0,108,"2973, 2907, 2749, 3357",1,3,3,2222
3,42535,20131,347,4,0,291,3,44,"30, 1515, 1760, 2932, 1287, 2615, 3727, 2450, ...",1,3,3,2222
4,42535,4325,539,6,0,303,0,45,"3104, 1772, 2029, 1274, 3915, 888, 1118, 3882,...",1,3,3,2222


In [11]:
# WARNING: This Cell takes a couple of minutes to compute; ~50GB RAM needed to compute without chunks

# Convert categorical variable into dummy/indicator variables
df_multi_hot = df_data_chunk1["categories"].str.get_dummies(',')
df_multi_hot

,1,1000,1001,1002,1003,1004,1006,1007,1008,1009,...,955,956,964,968,970,975,981,990,995,996
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
149996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
149997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
149998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# Combine df_data and df_multi_hot
#pd.merge(df_data_chunk1, df_multi_hot, how='inner', left_index=True, right_index=True)
df_combined = df_data_chunk1.join(df_multi_hot, how='inner')

# delete df_multi_hot to free up RAM
#del df_multi_hot

In [13]:
#df_combined.info()

In [14]:
df_combined.head()

,userID,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories,week,...,955,956,964,968,970,975,981,990,995,996
0,38769,3477,186,6,0,196,0,45,"74, 4109, 3867, 803, 4053",1,...,0,0,0,0,0,0,0,0,0,0
1,42535,30474,193,10,3,229,3,132,"3459, 3738, 679, 1628, 4072",1,...,0,0,0,0,0,0,0,0,0,0
2,42535,15833,1318,4,1,455,0,108,"2973, 2907, 2749, 3357",1,...,0,0,0,0,0,0,0,0,0,0
3,42535,20131,347,4,0,291,3,44,"30, 1515, 1760, 2932, 1287, 2615, 3727, 2450, ...",1,...,0,0,0,0,0,0,0,0,0,0
4,42535,4325,539,6,0,303,0,45,"3104, 1772, 2029, 1274, 3915, 888, 1118, 3882,...",1,...,0,0,0,0,0,0,0,0,0,0


In [15]:
#df_combined_chunk1 = df_combined.iloc[:500000,:]
col = df_combined.pop("week")
df_combined.insert(len(df_combined.columns), col.name, col)

#df_combined_chunk2 = df_combined.iloc[500000:,:]

In [16]:
df_combined.head()

,userID,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories,day,...,956,964,968,970,975,981,990,995,996,week
0,38769,3477,186,6,0,196,0,45,"74, 4109, 3867, 803, 4053",3,...,0,0,0,0,0,0,0,0,0,1
1,42535,30474,193,10,3,229,3,132,"3459, 3738, 679, 1628, 4072",3,...,0,0,0,0,0,0,0,0,0,1
2,42535,15833,1318,4,1,455,0,108,"2973, 2907, 2749, 3357",3,...,0,0,0,0,0,0,0,0,0,1
3,42535,20131,347,4,0,291,3,44,"30, 1515, 1760, 2932, 1287, 2615, 3727, 2450, ...",3,...,0,0,0,0,0,0,0,0,0,1
4,42535,4325,539,6,0,303,0,45,"3104, 1772, 2029, 1274, 3915, 888, 1118, 3882,...",3,...,0,0,0,0,0,0,0,0,0,1


In [17]:
#del df_data
#del df_data_chunk1
#del df_multi_hot
#del col

import gc

gc.collect()

17

In [18]:
# Check if we have any missing values
df_combined[df_combined.isnull().any(axis=1)]

,userID,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories,day,...,956,964,968,970,975,981,990,995,996,week


In [19]:
"""
For Testing purposes

# Read csv
df_cat = pd.read_csv('E:\OneDrive\Arbeit\Repos\DMC2022\Kevin\csv\complete_dataset_cleaned_labeled.csv', sep='|')
df_cat["categories"] = df_cat["categories"].astype("string")

df_cat.drop('Unnamed: 0', axis=1, inplace=True)
df_cat.drop('date', axis=1, inplace=True)
df_cat.drop('userID', axis=1, inplace=True)
df_cat.drop('itemID', axis=1, inplace=True)
df_cat.drop('order', axis=1, inplace=True)
df_cat.drop('brand', axis=1, inplace=True)
df_cat.drop('feature_1', axis=1, inplace=True)
df_cat.drop('feature_2', axis=1, inplace=True)
df_cat.drop('feature_3', axis=1, inplace=True)
df_cat.drop('feature_4', axis=1, inplace=True)
df_cat.drop('feature_5', axis=1, inplace=True)
df_cat.drop('week', axis=1, inplace=True)

df_cat = df_cat.head(3)
df_cat
"""

'\nFor Testing purposes\n\n# Read csv\ndf_cat = pd.read_csv(\'E:\\OneDrive\\Arbeit\\Repos\\DMC2022\\Kevin\\csv\\complete_dataset_cleaned_labeled.csv\', sep=\'|\')\ndf_cat["categories"] = df_cat["categories"].astype("string")\n\ndf_cat.drop(\'Unnamed: 0\', axis=1, inplace=True)\ndf_cat.drop(\'date\', axis=1, inplace=True)\ndf_cat.drop(\'userID\', axis=1, inplace=True)\ndf_cat.drop(\'itemID\', axis=1, inplace=True)\ndf_cat.drop(\'order\', axis=1, inplace=True)\ndf_cat.drop(\'brand\', axis=1, inplace=True)\ndf_cat.drop(\'feature_1\', axis=1, inplace=True)\ndf_cat.drop(\'feature_2\', axis=1, inplace=True)\ndf_cat.drop(\'feature_3\', axis=1, inplace=True)\ndf_cat.drop(\'feature_4\', axis=1, inplace=True)\ndf_cat.drop(\'feature_5\', axis=1, inplace=True)\ndf_cat.drop(\'week\', axis=1, inplace=True)\n\ndf_cat = df_cat.head(3)\ndf_cat\n'

### Datatypes for XGBoost

XGBoost natively supports continuous data but not categorical data. In order to use categorical data with XGBoost, we have to use One-Hot-Encoding which converts a column of categorical values into muliple columns of binary values.

# Modeling

In [20]:
df_combined.head()

,userID,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories,day,...,956,964,968,970,975,981,990,995,996,week
0,38769,3477,186,6,0,196,0,45,"74, 4109, 3867, 803, 4053",3,...,0,0,0,0,0,0,0,0,0,1
1,42535,30474,193,10,3,229,3,132,"3459, 3738, 679, 1628, 4072",3,...,0,0,0,0,0,0,0,0,0,1
2,42535,15833,1318,4,1,455,0,108,"2973, 2907, 2749, 3357",3,...,0,0,0,0,0,0,0,0,0,1
3,42535,20131,347,4,0,291,3,44,"30, 1515, 1760, 2932, 1287, 2615, 3727, 2450, ...",3,...,0,0,0,0,0,0,0,0,0,1
4,42535,4325,539,6,0,303,0,45,"3104, 1772, 2029, 1274, 3915, 888, 1118, 3882,...",3,...,0,0,0,0,0,0,0,0,0,1


In [21]:
df_combined.drop('categories', axis=1, inplace=True)

In [22]:
df_combined.head()

,userID,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,day,month,...,956,964,968,970,975,981,990,995,996,week
0,38769,3477,186,6,0,196,0,45,3,3,...,0,0,0,0,0,0,0,0,0,1
1,42535,30474,193,10,3,229,3,132,3,3,...,0,0,0,0,0,0,0,0,0,1
2,42535,15833,1318,4,1,455,0,108,3,3,...,0,0,0,0,0,0,0,0,0,1
3,42535,20131,347,4,0,291,3,44,3,3,...,0,0,0,0,0,0,0,0,0,1
4,42535,4325,539,6,0,303,0,45,3,3,...,0,0,0,0,0,0,0,0,0,1


In [23]:
# Comma is being used to extract a specific column from a 2D array.
# X = data.iloc[:,:-1]
# X = all rows, all columns except the last one 

X = df_combined.iloc[:,0:-1]
X

,userID,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,day,month,...,955,956,964,968,970,975,981,990,995,996
0,38769,3477,186,6,0,196,0,45,3,3,...,0,0,0,0,0,0,0,0,0,0
1,42535,30474,193,10,3,229,3,132,3,3,...,0,0,0,0,0,0,0,0,0,0
2,42535,15833,1318,4,1,455,0,108,3,3,...,0,0,0,0,0,0,0,0,0,0
3,42535,20131,347,4,0,291,3,44,3,3,...,0,0,0,0,0,0,0,0,0,0
4,42535,4325,539,6,0,303,0,45,3,3,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,27465,4284,1222,6,0,502,0,185,3,3,...,0,0,0,0,0,0,0,0,0,0
149996,27465,4284,1222,6,0,502,0,185,25,6,...,0,0,0,0,0,0,0,0,0,0
149997,27465,4284,1222,6,0,502,0,185,23,7,...,0,0,0,0,0,0,0,0,0,0
149998,27465,4284,1222,6,0,502,0,185,19,10,...,0,0,0,0,0,0,0,0,0,0


In [24]:
y = df_combined.iloc[:,-1]
y

0         1
1         1
2         1
3         1
4         1
         ..
149995    4
149996    4
149997    3
149998    3
149999    4
Name: week, Length: 150000, dtype: int64

In [25]:
# Split training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [26]:
data_dmatrix = xgb.DMatrix(data=X, label=y)

In [27]:
model = XGBClassifier()
model.fit(X_train, y_train)

C:\Users\LEAND\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:01:18] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [28]:
model.score(X,y)

0.38192

In [29]:
model = XGBClassifier(learning_rate=0.1)
model.fit(X_train, y_train)

C:\Users\LEAND\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:09:45] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [30]:
model.score(X,y)

0.36017333333333335

In [31]:
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 34.96%


In [32]:
#model = XGBClassifier(tree_method='gpu_hist', gpu_id=0)
#model.fit(X_train, y_train)

In [33]:
#xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
#                max_depth = 5, alpha = 10, n_estimators = 10)

In [34]:
#xg_reg.fit(X_train,y_train)

#preds = xg_reg.predict(X_test)

In [35]:
# Je größer der RMSE ist, desto schlechter ist die Anpassung des Modells.
#rmse = np.sqrt(mean_squared_error(y_test, preds))
#print("RMSE: %f" % (rmse))

In [36]:
#!conda list

In [37]:
conda install -c nvidia -c rapidsai py-xgboost


Note: you may need to restart the kernel to use updated packages.


Traceback (most recent call last):
  File "C:\Users\LEAND\anaconda3\Scripts\conda-script.py", line 11, in <module>
    from conda.cli import main
ModuleNotFoundError: No module named 'conda'
